## Домашня работа 1.

*Гриневская Катя, БКЛ182*

In [14]:
# импорты


import os
import pandas as pd
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize
from summa import keywords
import RAKE
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
texts = ['baranov', 'belikov', 'detkova', 'kustovagi']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Задание 1.
__Подготовить мини-корпус (не меньше 4 текстов, примерный общий объём - 3-5 тысяч токенов) с разметкой ключевых слов. Предполагается, что вы найдете источник текстов, в котором уже выделены ключевые слова. Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова.__

Корпус - 4 статьи с конференции Диалог 2020 года:
- "Динамика стиля русской письменной речи XIX века: корпусный эксперимент" (Баранов А. Н., Добровольский Д. О.)
- "Интернет-корпус как инструмент лингвистических исследований: дифференциальность, авторизация, тематические смещения (или корпусы, которым так хочется верить)" (Беликов В., Селегей В., Селегей Д.)
- "Дифференциальные семантические скетчи для русскоязычных интернет-корпусов" (Деткова Ю., Новицкий В., Петрова М., Селегей В.)
- "Семантические эффекты времени во вводных конструкциях с ментальными глаголами" (Кустова Г.И.)


In [63]:
# Если на локальном сервере

# в папке "corpus", лежащей в одной папке с кодом программы,
# должны лежать тексты статей

def get_docs_loc(dir_name):
    corpus = []
    curr_dir = os.getcwd()
    files_dir = os.path.join(curr_dir, dir_name)
    for root, dirs, files in os.walk(files_dir):
        for name in files:
            fpath = os.path.join(root, name)
            with open(fpath, 'r', encoding='utf-8') as f:
                text = f.read().replace('\n\n', ' ').replace('\n', ' ')
                corpus.append(text)
    return corpus

In [17]:
# Если в колабе

def get_docs_col():
    corpus = []
    files = ['baranovanplusdobrovol-skijdo-009.txt',
             'belikovvplusetal-060.txt',
             'detkovajplusetal-059.txt',
             'kustovagi-065.txt']
    for name in files:
        with open(name, 'r', encoding='utf-8') as f:
            text = f.read().replace('\n\n', ' ').replace('\n', ' ')
            corpus.append(text)
    return corpus

In [18]:
# Подсчёт объёма корпуса

def len_tokens(corp):
    num_of_toks = 0
    for t in corp:
        t = t.split()
        num_of_toks += len(t)
    return num_of_toks

In [19]:
# corpus = get_docs_loc('corpus')
corpus = get_docs_col()
print('Общий объём корпуса: ', len_tokens(corpus))

Общий объём корпуса:  13639


### Задание 2.
__Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой. Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной).__

##### Ключевые слова из текстов статей

In [20]:
kw_baranov_text = ['дискурсивные слова', 'авторский стиль', 'корпусный подход',
                   'динамика стиля']
kw_kustovagi_text = ['парентетические конструкции', 'ментальные глаголы',
                     'категория времени', 'категория лица']
kw_belikov_text = ['дифференциальный корпус', 'социолингвистическая вариативность',
                   'тематические смещения', 'ГИКРЯ']
kw_detkova_text = ['семантические скетчи', 'автоматическая семантическая разметка',
                   'семантические классы', 'глубинные позиции',
                   'лексическая сочетаемость', 'ГИКРЯ']

##### Ключевые слова, выделенные самостоятельно

Сюда я выписывала слова, которых нет в ключевых словах, указанных авторами.

In [21]:
my_kw_baranov = ['лингво-статистическая модель', 'пропозиция', 'вводные слова',
                 'дискурсивная единица', 'художественная литература']
my_kw_kustovagi = ['статус пропозиции', 'редукция', 'конструкция', 'вводная конструкция']
my_kw_belikov = ['контекст', 'автоматическая фильтрация', 'авторский стиль']
my_kw_detkova = ['семантическая модель', 'парсер'
                 'лексические классы']

##### Эталон

Наиболее удачным набором ключевых слов, на мой взгляд, является объединение ключевых слов, выделенных авторами статьи, и ключевых слов, выделенных самостоятельно.

In [92]:
kw_baranov = my_kw_baranov + kw_baranov_text
print(kw_baranov)
kw_kustovagi = my_kw_kustovagi + kw_kustovagi_text
print(kw_kustovagi)
kw_belikov = my_kw_belikov + kw_belikov_text
print(kw_belikov)
kw_detkova = my_kw_detkova + kw_detkova_text
print(kw_detkova)
kw_temp = [kw_baranov, kw_kustovagi, kw_belikov, kw_detkova]

['лингво-статистическая модель', 'пропозиция', 'вводные слова', 'дискурсивная единица', 'художественная литература', 'дискурсивные слова', 'авторский стиль', 'корпусный подход', 'динамика стиля']
['статус пропозиции', 'редукция', 'конструкция', 'вводная конструкция', 'парентетические конструкции', 'ментальные глаголы', 'категория времени', 'категория лица']
['контекст', 'автоматическая фильтрация', 'авторский стиль', 'дифференциальный корпус', 'социолингвистическая вариативность', 'тематические смещения', 'ГИКРЯ']
['семантическая модель', 'парсерлексические классы', 'семантические скетчи', 'автоматическая семантическая разметка', 'семантические классы', 'глубинные позиции', 'лексическая сочетаемость', 'ГИКРЯ']


### Задание 3.
__Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf*idf, OKAPI BM25, ...)__

In [23]:
# Лемматизация текстов

def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [24]:
# Препроцессинг

def preproc(raw_text):
    text = raw_text.lower().replace('\n\n', ' ').replace('\n', ' ').split()
    f = lambda w: m.parse(w.strip(punctuation))[0].normal_form
    clean_text = ' '.join([f(w) for w in text if w.isalpha() and w not in stop])
    return clean_text

In [30]:
def rake(corp):
    kw_all = []
    for text in corp:
        rake = RAKE.Rake(stop)
        kw = rake.run(normalize_text(text), maxWords=3, minFrequency=2)
        kw_all.append(kw)
    return kw_all

In [31]:
def text_rank(corp):
    kw_all = []
    for text in corp:
        kw = keywords.keywords(normalize_text(text), language='russian',
                               additional_stopwords=stop, scores=True)
        kw_all.append(kw)
    return kw_all

In [32]:
def tfidf(corp):
    prep_corp = [preproc(t) for t in corp]
    tfidf = TfidfVectorizer(max_features=300, ngram_range=(1, 2))
    X = tfidf.fit_transform(prep_corp)
    terms = tfidf.get_feature_names()
    df = pd.DataFrame(X.todense().tolist(), columns=terms, index=texts)
    kw_all = []
    for t in texts:
        n = pd.Series(df.loc[t])
        ser = n[n > 0].sort_values(ascending=False)
        kw_tuples = ser.items()
        kw_all.append(list(kw_tuples))
    return kw_all

In [33]:
# посмотрим, что получилось

print('RAKE')
kw_rake = rake(corpus)
for k in kw_rake:
    print(k)
print('TEXT-RANK')
kw_rank = text_rank(corpus)
for k in kw_rank:
    print(k)
print('TF-IDF')
kw_tfidf = tfidf(corpus)
for k in kw_tfidf:
    print(k)

RAKE
[('описывать положение дело', 9.0), ('эпи- стемичёска модальность', 9.0), ('салтыков-щедрин', 5.071428571428571), ('по-видимому', 5.0), ('xix век', 4.678571428571429), ('общий тенденция', 4.666666666666667), ('синтаксический конструкция', 4.5), ('иной слово', 4.461538461538462), ('частотный распределение', 4.4), ('всякий сомнение', 4.333333333333334), ('дискурсивный употребление', 4.111111111111111), ('дискурсивный практика', 4.111111111111111), ('салтык-щедрина', 4.0), ('мельникова-печерский', 4.0), ('весь вероятность', 3.666666666666667), ('проявляться', 2.0), ('думать', 2.0), ('конструкция', 2.0), ('это', 1.875), ('также', 1.75), ('смысл', 1.75), ('некоторый', 1.75), ('год', 1.7142857142857142), ('н', 1.7142857142857142), ('весь', 1.6666666666666667), ('выражение', 1.6666666666666667), ('подтверждать', 1.6666666666666667), ('близкий', 1.6), ('который', 1.5714285714285714), ('отношение', 1.5), ('вероятно', 1.5), ('однако', 1.5), ('казаться', 1.375), ('например', 1.33333333333333

### Задание 4.
__Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun). Применить эти фильтры к спискам ключевых слов.__

In [82]:
# разметка

def tags(raw):
    pos_raw = ''
    for r in raw.split():
        if m.parse(r)[0].tag.POS:
            pos_raw += r + '_' + m.parse(r)[0].tag.POS + ' '
        else:
            pos_raw = ''
    return pos_raw


In [60]:
# фильтрация

def filter(phrase, template):
    tags = []
    words = []
    for w in phrase.split():
        word = w.split('_')[0]
        words.append(word)
        tag = w.split('_')[1]
        tags.append(tag)
    words = ' '.join(words)
    tags = '+'.join(tags)
    if tags in template:
        return words

In [59]:
temp = ['ADJF+NOUN', 'ADJS+NOUN', 'NOUN', 'NOUN+NOUN', 'ADJF+ADJF+NOUN']

In [86]:
kws = [kw_rake, kw_rank, kw_tfidf]
kws_filtered = []
for method in kws:
    method_filtered = []
    for kw in method:
        kw_filtered = []
        for phrase in kw:
            pos_phrase = tags(preproc(phrase[0]))
            filtered = filter(pos_phrase, temp)
            if filtered:
                if len(filtered) > 1:
                    kw_filtered.append(filtered)
        method_filtered.append(kw_filtered)
    kws_filtered.append(method_filtered)

In [89]:
kw_rake_filtered, kw_rank_filtered, kw_tfidf_filtered = kws_filtered

In [111]:
# посмотрим, что получилось после фильтрации

for i in kws_filtered:
    for j in i:
        print(j[:10])

['стемичёска модальность', 'век', 'общий тенденция', 'синтаксический конструкция', 'иной слово', 'частотный распределение', 'всякий сомнение', 'дискурсивный употребление', 'дискурсивный практика', 'весь вероятность']
['динамический дедубликация', 'частотный словарь', 'сигнатурный фильтрация', 'социальный сеть', 'регион автор', 'корпус', 'очистка', 'фильтрация', 'пример', 'датасет']
['коллокация глагол', 'построение скетч', 'семантический скетч', 'частота встречаемость', 'семантический класс', 'грамматический отношение', 'лексический класс', 'каждый слово', 'глубинный позиция', 'выбор ск']
['прошедшее время', 'вводный слово', 'глагол речь', 'момент речь', 'последующий текст', 'настоящий время', 'неправильный предположение', 'вводный конструкция', 'глагол мнение', 'главный клауза']
['слово', 'автор', 'автор', 'тексто', 'частотность', 'тенденция', 'группа', 'форма выражение', 'корпус', 'корпус']
['текст', 'результат', 'слово', 'таблица', 'датасет', 'динамический дедубликация', 'задача', '

### Задание 5.
__Оценить точность, полноту, F-меру выбранных методов относительно эталона: с учётом морфосинтаксических шаблонов и без них.__

In [101]:
# точность, полность, f-мера

def precision_recall_f(predicted, template):
    conj = list(set(predicted) & set(template))
    precision = len(conj)/len(predicted)
    recall = len(conj)/len(template)
    if precision + recall == 0:
        f = 0
    else:
        f = (2 * precision * recall) / (precision + recall)
    return precision, recall, f

In [93]:
# лемматизация эталонных ключевых слов

kw_lemmatized = []
for art in kw_temp:
    art_lem = []
    for kw in art:
        kw_lem = preproc(kw)
        art_lem.append(kw_lem)
    kw_lemmatized.append(art_lem)

In [107]:
# подсчёт и вывод точности, полноты и f-меры

def compute_and_show(filtered_or_not):
    df_dict = {}
    for i, method in enumerate(filtered_or_not):
        precisions = []
        recalls = []
        fs = []
        for j, kw in enumerate(method):
            precision, recall, f = precision_recall_f(kw, kw_lemmatized[j])
            precisions.append(precision)
            recalls.append(recall)
            fs.append(f)
        df_dict[methods[i]+'_precision'] = precisions
        df_dict[methods[i]+'_recall'] = recall
        df_dict[methods[i]+'_f-score'] = fs
    return df_dict

In [94]:
# удаление значений, равныx цифрам из предсказанных не отфильтрованных ключевых слов

kws_not_filtered = []
for i in kws:
    b = []
    for j in i:
        a = j[0]
        b.append(a)
    kws_not_filtered.append(b)

In [108]:
methods = ['RAKE', 'TEXT_RANK', 'TF-IDF']
print('C фильтрами')
pd.DataFrame(compute_and_show(kws_filtered))

C фильтрами


,RAKE_precision,RAKE_recall,RAKE_f-score,TEXT_RANK_precision,TEXT_RANK_recall,TEXT_RANK_f-score,TF-IDF_precision,TF-IDF_recall,TF-IDF_f-score
0,0.0,0.0,0,0.0,0.0,0,0.043478,0.0,0.079208
1,0.0,0.0,0,0.0,0.0,0,0.000000,0.0,0.000000
2,0.0,0.0,0,0.0,0.0,0,0.010526,0.0,0.019608
3,0.0,0.0,0,0.0,0.0,0,0.000000,0.0,0.000000


In [109]:
print('БЕЗ фильтров')
pd.DataFrame(compute_and_show(kws_not_filtered))

БЕЗ фильтров


,RAKE_precision,RAKE_recall,RAKE_f-score,TEXT_RANK_precision,TEXT_RANK_recall,TEXT_RANK_f-score,TF-IDF_precision,TF-IDF_recall,TF-IDF_f-score
0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
1,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
2,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
3,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0


### Задание 6.
__Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется); предложить свои методы решения этих проблем.__

Я заметила следующие ошибки:

1.   В методе Rake, например, выделяется слишком много ключевых слов. Засчёт этого оценка у этого метода получается хужею Решить можно ограничением на скоры ключевых слов. 
2.   Моя фильтрация, кажется, оставляет возможность ключевым словам повторяться. Вероятно, это тоже ухудшает качество работы. Решение - убрать дубликаты.
3.   Часто выделяются просто частотные слова: *век*, *датасет*, *слово*, *пример*. Такие слова есть во всех текстах, тут можно было бы расширить список стоп-слов или исключать список популярных для тематики слов из документов.
4.   Метод TF-IDF выделяет слова, частотные для документа, не частотные для остальных документов коллекции, а потому слово *петя*, например, вылезло в слова с самым большим скором. Думаю, более тщательная предобработка могла бы с этим помочь. С другой стороны, просто выкинуть именованные сущности - не очень хороший вариант (ведь они часто могут быть значимы).
5.   Аббревиатуры не попались нигде. Возможно, это специфика выбранных текстов, а может, дело именно в том, что они являются аббревиатурами и пасрсеру сложно определить, какой частеречный тег им присвоить.

